<a href="https://colab.research.google.com/github/CCMunozB/Patrones/blob/main/10-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de texto con transformers

In [ ]:
!pip install datasets transformers evaluate accelerate xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from datasets import load_dataset # Manejo de conjuntos de datos (como pandas)
import evaluate # Métricas de rendimiento
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, BertModel, pipeline # Modelamiento con Transformers
import numpy as np

Importamos un conjunto de datos de diagnósticos médicos en español etiquetados como dentales y no-dentales.

In [ ]:
spanish_diagnostics = load_dataset('fvillena/spanish_diagnostics')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset spanish_diagnostics downloaded and prepared to /root/.cache/huggingface/datasets/fvillena___spanish_diagnostics/default/0.0.0/45c176cea64580ea9631f78c2867a657ede368597681e5337e9f1c976e4e84ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
spanish_diagnostics["train"].features


{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_dental', 'dental'], id=None)}

Vamos a medir el rendimiento con accuracy porque nuestro conjunto de datos está balanceado.

In [ ]:
metric = evaluate.load('accuracy')

Segmentamos los string de texto usando el tokenizador específico del modeo de Transformers que vamos a usar.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Tokenizamos el texto

In [ ]:
tokenized_spanish_diagnostics = spanish_diagnostics.map(preprocess_function, batched=True)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
id2label = {0: "not_dental", 1: "dental"}
label2id = {"not_dental": 0, "dental": 1}

Importamos un modelo basado en BERT que fue entrenado con un conjunto de datos en múltiples lenguajes.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Configuramos el entrenador de nuestro modelo

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    max_steps=500,
    evaluation_strategy="steps"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer = tokenizer,
    train_dataset=tokenized_spanish_diagnostics["train"],
    eval_dataset=tokenized_spanish_diagnostics["test"].shuffle(seed=11).select(range(1000)),
    compute_metrics=compute_metrics
)

Entrenamos el modelo

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.509000,0.359033,0.863000


TrainOutput(global_step=500, training_loss=0.5089598083496094, metrics={'train_runtime': 96.2138, 'train_samples_per_second': 41.574, 'train_steps_per_second': 5.197, 'total_flos': 152415301037760.0, 'train_loss': 0.5089598083496094, 'epoch': 0.06})

Probamos el modelo con ejemplos inventados por nosotros.

In [ ]:
classifier = pipeline("text-classification", model = model, tokenizer=tokenizer, device=0)

In [ ]:
classifier(["fractura de tobillo","caries dentinaria"])

[{'label': 'dental', 'score': 0.7107235789299011},
 {'label': 'not_dental', 'score': 0.8194347023963928}]

## Actividad 1

Usted acaba de ajustar un predictor de la etiqueta dental utilizando un modelo de lenguaje multilenguaje. Utilice un modelo de lenguaje ajustado para el lenguaje Español y vea si el rendimiento del modelo mejora.

Acá puede explorar muchos modelos de lenguaje: https://huggingface.co/models

## Actividad 2:

Nosotros acabamos de realizar un ajuste fino de un modelo de lenguaje, que significa agregar capas río abajo en la arquitectura del modelo, para poder resolver la tarea de clasificación de texto. Pero la salida del modelo de lenguaje no es una clasificación, sino que una secuencia de embeddings contextualizada.

Cargue un modelo de lenguaje basado en BERT (clase `transformers.BertModel`) junto con su tokenizador.

1. Tokenice un texto y explore cuántos tokens se detectaron. Por qué la cantidad de tokens puede ser inconsistente con la cantidad de palabras?
2. Pásele un texto tokenizado al modelo, explore la salida modelo (atributo `last_hidden_state` de la clase `transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions`) y vea cuántas dimensiones tiene esa salida. Por qué tiene esas dimensiones esa salida?